Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
#from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.835742
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.167509
Minibatch accuracy: 18.8%
Validation accuracy: 19.3%
Minibatch loss at step 100: 1.199223
Minibatch accuracy: 62.5%
Validation accuracy: 61.9%
Minibatch loss at step 150: 1.077612
Minibatch accuracy: 68.8%
Validation accuracy: 71.9%
Minibatch loss at step 200: 0.384534
Minibatch accuracy: 93.8%
Validation accuracy: 75.1%
Minibatch loss at step 250: 0.596877
Minibatch accuracy: 93.8%
Validation accuracy: 75.7%
Minibatch loss at step 300: 0.715510
Minibatch accuracy: 68.8%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.537048
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 400: 1.137465
Minibatch accuracy: 62.5%
Validation accuracy: 79.4%
Minibatch loss at step 450: 1.092949
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.712503
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
graph = tf.Graph()

with graph.as_default():
    
    #i nput data: at the CNN, we have depth in the layer, at first layer, depth means channels
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # variables
    #layer1 is the convolutional layers    (filter size :  [patch_size, patch_size, original_depth, filter_depth]
    layer1_weights = tf.Variable(tf.truncated_normal(shape=[patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(shape=[depth]))
    #layer2 is the convolutional layers
    layer2_weigths = tf.Variable(tf.truncated_normal(shape=[patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.zeros(shape=[depth]))
    #layer3 is the fully conected layer (hidden layer)
    layer3_weights = tf.Variable(tf.truncated_normal(shape=[image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #image_size // 4 * image_size // 4 * depth :7*7*depth
    layer3_biases = tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    #layer4 is output layer
    layer4_weights = tf.Variable(tf.truncated_normal(shape=[num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0,shape=[num_labels]))
        
    # model
    def model(data):
        # first convolutional operation
        conv1 = tf.nn.relu(tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME'))+layer1_biases
        pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        # second convolutional operation
        conv2 = tf.nn.relu(tf.nn.conv2d(pool1, layer2_weigths, [1, 1, 1, 1], padding='SAME'))+layer2_biases
        pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        # get the second layer'shape (n, l, l, depth)
        shape = pool2.get_shape().as_list()
        # reshape the output of convolution layers
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        # fully connected layer
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # output layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # training compution
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # predictions for the training, validation, test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

It is hard to create CNN as same as LeNet5 which has 8 layers. For other words, it will spend a lot of time to trianing LeNet5 model for my computer(core i7, GT645). Maybe I need 2 weeks and do nothing without training LeNet5 model. So I will use some methods I have learned at Lesson 3 like Dropout, Regulazition and adjust the learning rate. I will increase the training steps to 3001. It perfomace better than above.

In [12]:
keep_prob=0.5
alpha=0.001

graph = tf.Graph()

with graph.as_default():
    
    #i nput data: at the CNN, we have depth in the layer, at first layer, depth means channels
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # variables
    #layer1 is the convolutional layers    (filter size :  [patch_size, patch_size, original_depth, filter_depth]
    layer1_weights = tf.Variable(tf.truncated_normal(shape=[patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(shape=[depth]))
    #layer2 is the convolutional layers
    layer2_weigths = tf.Variable(tf.truncated_normal(shape=[patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.zeros(shape=[depth]))
    #layer3 is the fully conected layer (hidden layer)
    layer3_weights = tf.Variable(tf.truncated_normal(shape=[image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #image_size // 4 * image_size // 4 * depth :7*7*depth
    layer3_biases = tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    #layer4 is output layer
    layer4_weights = tf.Variable(tf.truncated_normal(shape=[num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    global_step=tf.Variable(0)
        
    # model
    def model(data):
        # first convolutional operation
        conv1 = tf.nn.relu(tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME'))+layer1_biases
        pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        # second convolutional operation
        conv2 = tf.nn.relu(tf.nn.conv2d(pool1, layer2_weigths, [1, 1, 1, 1], padding='SAME'))+layer2_biases
        pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        # get the second layer'shape (n, l, l, depth)
        shape = pool2.get_shape().as_list()
        # reshape the output of convolution layers
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        # fully connected layer
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        # output layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # training compution
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss = loss + alpha*(tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                         tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases))
    # optimizer
    learning_rate = tf.train.exponential_decay(1e-1,global_step,decay_steps=100,decay_rate=0.9,staircase=True)
    #means very decay_steps after, learning_rate will be update to learning_rate*decay_rate
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step)
    
    # predictions for the training, validation, test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 3001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print 'Initialized'
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)        
        batch_data = train_dataset[offset:(offset + batch_size),:,:,:]
        batch_labels = train_labels[offset:(offset + batch_size),:] 
        
        feed_dict={tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, prediction = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500) == 0:
            print 'Minibatch loss at step %s : %s' % (step, l)
            print 'Minibatch accuracy : %s' % accuracy(prediction, batch_labels)
            print 'validation accuracy : %s' % accuracy(valid_prediction.eval(), valid_labels)
    print 'test accuracy : %s' % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.0837026
Minibatch accuracy : 12.5
validation accuracy : 11.58
Minibatch loss at step 500 : 0.6708275
Minibatch accuracy : 87.5
validation accuracy : 77.72
Minibatch loss at step 1000 : 0.6875799
Minibatch accuracy : 87.5
validation accuracy : 81.31
Minibatch loss at step 1500 : 0.888713
Minibatch accuracy : 87.5
validation accuracy : 83.01
Minibatch loss at step 2000 : 1.1164069
Minibatch accuracy : 68.75
validation accuracy : 83.48
Minibatch loss at step 2500 : 0.3612612
Minibatch accuracy : 100.0
validation accuracy : 84.0
Minibatch loss at step 3000 : 0.8688301
Minibatch accuracy : 87.5
validation accuracy : 84.14
test accuracy : 91.04
